# return one Python dictionary containing all of the scraped data.
I couldn't figure out how to do this well, so I inserted each dictionary separately.  I think I have similar results, but we'll see.


In [496]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
master_mars_dict={}
from flatten_dict import flatten
from pprint import pprint
import pymongo

In [497]:
conn='mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db=client.marsDB

In [498]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'marsDB')

In [499]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [500]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [501]:
# mongo.db.mars_info[randomly_generated_number]
# gathered={
#     'news': news_dict, 
#     'images': images_list
# }

In [502]:
html = browser.html
soup = BeautifulSoup(html,'html.parser')
articles = soup.find_all('div',class_='list_text')
news_dict=[]

title_list=[]
para_list=[]

for article in articles:
    title = article.find('a').text.strip()
    title_list.append(title)
    para = article.find('div',class_='article_teaser_body').text.strip()
    para_list.append(para)
    news_pairs = {
    "news_title": title,
    "news_para": para
    }
        # Append hemisphere object to list
    news_dict.append(news_pairs)
print(news_dict)

[{'news_title': "NASA Readies Perseverance Mars Rover's Earthly Twin", 'news_para': "Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape."}, {'news_title': "NASA Engineers Checking InSight's Weather Sensors", 'news_para': 'An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.'}, {'news_title': "Follow NASA's Perseverance Rover in Real Time on Its Way to Mars", 'news_para': "A crisply rendered web application can show you where the agency's Mars 2020 mission is right now as it makes its way to the Red Planet for a Feb. 18, 2021, landing."}, {'news_title': 'NASA Establishes Board to Initially Review Mars Sample Return Plans', 'news_para': 'The board will assist with analysis of current plans and goals for one of the most difficult missions humanity has ever undertaken.'}, {'news_title': "NASA's

In [504]:
db.mars_all.insert_many(news_dict)

In [505]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [506]:
browser.click_link_by_id('full_image')

In [507]:
html = browser.html
soup = BeautifulSoup(html,'html.parser')

featured_image_url = soup.article.a['data-fancybox-href']
featured_image_url = (f'https://www.jpl.nasa.gov{featured_image_url}')
descrip =soup.h1.text.strip()
print(featured_image_url)
print(descrip)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19685_ip.jpg
Study: Third of Big Groundwater Basins in Distress


In [508]:
featured_image_dict=[]
featured_image_dict = [{
    "featured_image_url": featured_image_url,
    "description": descrip
    }]
print(featured_image_dict)
db.mars_all.insert_many(featured_image_dict)

[{'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19685_ip.jpg', 'description': 'Study: Third of Big Groundwater Basins in Distress'}]


In [487]:
url = 'https://space-facts.com/mars/'

In [488]:
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [489]:
df = tables[0]
df.columns = ['info_type', 'mars_fact']
# df = df.set_index('info_type')
df

,info_type,mars_fact
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [509]:
facts_dict = df.to_dict('records')
print(facts_dict)
db.mars_all.insert_many(facts_dict)

[{'info_type': 'Equatorial Diameter:', 'mars_fact': '6,792 km'}, {'info_type': 'Polar Diameter:', 'mars_fact': '6,752 km'}, {'info_type': 'Mass:', 'mars_fact': '6.39 × 10^23 kg (0.11 Earths)'}, {'info_type': 'Moons:', 'mars_fact': '2 (Phobos & Deimos)'}, {'info_type': 'Orbit Distance:', 'mars_fact': '227,943,824 km (1.38 AU)'}, {'info_type': 'Orbit Period:', 'mars_fact': '687 days (1.9 years)'}, {'info_type': 'Surface Temperature:', 'mars_fact': '-87 to -5 °C'}, {'info_type': 'First Record:', 'mars_fact': '2nd millennium BC'}, {'info_type': 'Recorded By:', 'mars_fact': 'Egyptian astronomers'}]


In [510]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)


In [511]:
# html = browser.html
# soup = BeautifulSoup(html,'html.parser')
# hemispheres = soup.find_all('div',class_='description')
# d={}

# hemi_image_link_list=[]
# hemi_image_list=[]
# hemi_title_list=[]
# i=0

# for hemisphere in hemispheres:
#     hemi_image = hemisphere.a['href']
#     hemi_image_list.append(hemi_image)
#     img_url = (f'https://astrogeology.usgs.gov{hemi_image}.tiff/full.jpg')
#     hemi_image_link_list.append(img_url)
#     title = hemisphere.h3.text.strip()
#     hemi_title_list.append(title)
#     d["dict"] = {"title":title, "img_url": img_url}

#     i=i+1

    
# print(hemi_image_list)
# print(hemi_title_list)
# print(hemi_image_link_list)
# # print(hemi_dictionary_1)

In [512]:
url = ("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")

browser.visit(url)

# Click the link, find the sample anchor, return the href
hemisphere_image_urls_dict = []
for i in range(4):
    # Find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    hemi_soup = BeautifulSoup(browser.html, "html.parser")

    # adding try/except for error handling
    try:
        title_elem = hemi_soup.find("h2", class_="title").get_text()
        sample_elem = hemi_soup.find("a", text="Sample").get("href")

    except AttributeError:
# Image error will return None, for better front-end handling
        title_elem = None
        sample_elem = None

    hemispheres = {
        "title": title_elem,
        "img_url": sample_elem
    }
    # Append hemisphere object to list
    hemisphere_image_urls_dict.append(hemispheres)
    # Finally, we navigate backwards
    browser.back()
print(hemisphere_image_urls_dict)
db.mars_all.insert_many(hemisphere_image_urls_dict)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [513]:
list_of_list_of_dicts = [news_dict,featured_image_dict,facts_dict,hemisphere_image_urls_dict]
list_of_list_of_dicts

[[{'news_title': "NASA Readies Perseverance Mars Rover's Earthly Twin",
   'news_para': "Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.",
   '_id': ObjectId('5f6221c0376e3a8a18daee0f')},
  {'news_title': "NASA Engineers Checking InSight's Weather Sensors",
   'news_para': 'An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.',
   '_id': ObjectId('5f6221c0376e3a8a18daee10')},
  {'news_title': "Follow NASA's Perseverance Rover in Real Time on Its Way to Mars",
   'news_para': "A crisply rendered web application can show you where the agency's Mars 2020 mission is right now as it makes its way to the Red Planet for a Feb. 18, 2021, landing.",
   '_id': ObjectId('5f6221c0376e3a8a18daee11')},
  {'news_title': 'NASA Establishes Board to Initially Review Mars Sample Return Plans',
   'news_pa